Note: This notebook assumes that "gmn_example3_visualise_outputs.ipynb" has been run.

This script visualises the extracted frames and lets the user interactively classify the frames as either having a contrail or not having a contrail.

This example also requires the installation of pyglet ("pip install pyglet") and ffmpeg (instructions for making this work with pyglet can be found [here](https://pyglet.readthedocs.io/en/latest/programming_guide/media.html#guide-supportedmedia-ffmpeg). 

In [1]:
import sys
sys.path.append("..")

import os
from glob import glob
import json

import tqdm

import pyglet
pyglet.options['search_local_libs'] = True

In [2]:
base_dir = '../data/gmn_extracted_flight_images/US0001'

In [3]:
class LabelUtils():
    def __init__(self):
        self.state = 'playing_video'
        self.counter = 0
        
width = 1280
height = 720
window = pyglet.window.Window(width, height, 'Simple Contrail Data Annotation Tool')

global state
label_utils = LabelUtils()

# creating a media player object
player = pyglet.media.Player()

@player.event
def end_of_video():
    label_utils.state = 'awaiting_input'

player.on_eos = end_of_video

# Create text to show for user instructions
playing_video_label = pyglet.text.Label('Press space to stop the video early and label it',
                          font_name='Times New Roman',
                          font_size=12, x=window.width//2, y=window.height * 0.99, color=(255, 0, 0),
                          anchor_x='center', anchor_y='center', width=window.width, multiline=True)

awaiting_input_label = pyglet.text.Label('Press the following keys to label the video:\n 1: Contrail (clearly visible) \n 2: Contrail (partly visible) \n 3: Contrail (barely visible) \n 5: No contrail observed (with good visiblility conditions) \n 6: No contrail observed (with okay visiblility conditions) \n 7: No contrail observed (with bad visibility conditions) \n 9: Replay Video \n 0: Skip video (e.g. if the flight is not in frame and should not be part of the dataset)',
                          font_name='Times New Roman',
                          font_size=18, x=window.width//2, y=window.height//2,
                          anchor_x='center', anchor_y='center', width=window.width, multiline=True)

def write_label_to_file_and_step(video_dir, label, label_utils, progress_bar, player):
    base_dir = os.path.dirname(os.path.dirname(video_dir))
    label_file = os.path.join(base_dir, 'metadata', 'label.json')
    with open(label_file, 'w') as f:
        json.dump({'label': label}, f)

    label_utils.state = 'playing_video'
    label_utils.counter += 1
    progress_bar.update(1)
    player.next_source()

In [ ]:
video_files = glob(os.path.join(base_dir, '*/*/debug/*.mp4'))

filtered_video_files = []

for video_file in video_files:
    if os.path.exists(os.path.join(os.path.dirname(os.path.dirname(video_file)), 'metadata', 'label.json')):
        print(f'File {video_file} already labelled, skipping')
    else:
        filtered_video_files.append(video_file)

if len(filtered_video_files) == 0:
    print('No video files found, make sure that "gmn_example2_visualise_outputs.ipynb" has been run')

for video_file in filtered_video_files:
    # load the media from the source
    media_load = pyglet.media.load(video_file)
 
    # add this media in the queue
    player.queue(media_load)

# play the video
player.play()

progress_bar = tqdm.tqdm(total=len(filtered_video_files))

# on draw event 
@window.event 
def on_draw(): 
    window.clear() 

    if label_utils.state == 'playing_video':
        # clear the window 
        player.seek_next_frame()
        player.texture.blit(0, 0)
        playing_video_label.draw()
    elif label_utils.state == 'awaiting_input':
        awaiting_input_label.draw()

# key press event     
@window.event 
def on_key_press(symbol, modifier): 
   
    # key "C" get press 
    if label_utils.state == 'awaiting_input':
        if symbol == pyglet.window.key._1: 
            print(f'Classified {filtered_video_files[label_utils.counter]} as clearly_visible')
            write_label_to_file_and_step(filtered_video_files[label_utils.counter], 'clearly_visible', label_utils, progress_bar, player)
        elif symbol == pyglet.window.key._2:
            print(f'Classified {filtered_video_files[label_utils.counter]} as partly_visible')
            write_label_to_file_and_step(filtered_video_files[label_utils.counter], 'partly_visible', label_utils, progress_bar, player)
        elif symbol == pyglet.window.key._3:
            print(f'Classified {filtered_video_files[label_utils.counter]} as barely_visible')
            write_label_to_file_and_step(filtered_video_files[label_utils.counter], 'barely_visible', label_utils, progress_bar, player)
        elif symbol == pyglet.window.key._5:
            print(f'Classified {filtered_video_files[label_utils.counter]} as no_contrail (good visibility)')
            write_label_to_file_and_step(filtered_video_files[label_utils.counter], 'no_contrail_good_visibility', label_utils, progress_bar, player)
        elif symbol == pyglet.window.key._6:
            print(f'Classified {filtered_video_files[label_utils.counter]} as no_contrail (okay visibility)')
            write_label_to_file_and_step(filtered_video_files[label_utils.counter], 'no_contrail_okay_visibility', label_utils, progress_bar, player)
        elif symbol == pyglet.window.key._7:
            print(f'Classified {filtered_video_files[label_utils.counter]} as no_contrail (bad visibility)')
            write_label_to_file_and_step(filtered_video_files[label_utils.counter], 'no_contrail_bad_visibility', label_utils, progress_bar, player)
        elif symbol == pyglet.window.key._9:
            player.seek(0)
            label_utils.state = 'playing_video'
            print(f'Replaying video {filtered_video_files[label_utils.counter]}')
        elif symbol == pyglet.window.key._0:
            print(f'Skipping {filtered_video_files[label_utils.counter]}')
            write_label_to_file_and_step(filtered_video_files[label_utils.counter], 'skip', label_utils, progress_bar, player)
        else:
            pass
    elif label_utils.state == 'playing_video':
        if symbol == pyglet.window.key.SPACE:
            label_utils.state = 'awaiting_input'

# run the pyglet application
pyglet.app.run(interval=1/60)